goal: train 10 machine learning predictors: one predictor for each function from the ten protein function categories ("DNA, RNA and nucleotide metabolism", "tail", "head and packaging", "other", "lysis", "connector", "transcription regulation", "moron, auxiliary metabolic gene and host takeover", "unknown function", "integration and excision") 

predictor input: protein features; output: labels (0/1) representing whether the protein serves the specific function

dataset: 360,413 seqs in total - 60% for training, 20% for validation, 20% for testing 
** subset the negatives (samples whose label is 0) and make it about 5-10 times of the positives
** change: split 20% for testing first, then train:val = 8:2

-use clustering results to avoid spliting protein seqs in the same cluster (maybe use GroupShuffleSplit from sklearn)
** change: for each predictor, cluster on the positive dataset (those with label "1") alone, and do nothing to negative dataset. use the clustering result while splitting dataset to ensure seqs in the same cluster in positive dataset is not splitted into different sets of train/val/test. I already have protein distances generated from diamond blastp in unique_diamond_results.daa, and would like to get the clustering result by select a cutoff of 100bitscore and use that to read into a graph with networkx and then extract subgraphs which then will be the clusters.

** change: add learning curve for training XGBoost

** change: try the 100 versions of threshold from 0.01 - 1

the results are printed as text.  
** change: not to print them out but to save it to results/predictor/{function_name}.txt


dataset:  
2,318,538 seqs in the original dataset  
927,040 seqs after dropping pcat "unknown_no_hit"  
360,413 unique seqs after dropping duplicated seqs  

features: 1711-dim

subset the negatives;  
- for each function, cluster on the positive dataset  
- try threshold from 0.01 to 1  
- [o] change the dataset splitting strategy  
- [o] model param : is_imbalanced  
- add learning curve (could be find from XGBoost)  
- n_estimators: increased to 10k; may need GPU

# training script

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict, Any
import joblib
import os
import json

In [3]:
def get_df(function_name: str):
    ids = pd.read_csv(f"../dataset/pcat/{function_name}.csv")
    features = pd.read_parquet("../dataset/protein_features_unique.pa")
    features["label"] = features["id"].isin(ids["name"]).astype(int)
    features = features.drop(columns=["md5"])
    return features

In [4]:
def subset_negatives(
    df: pd.DataFrame, label_col: str = "label", ratio: int = 5, random_state: int = 42
) -> pd.DataFrame:
    """
    Subset the negatives to be `ratio` times the number of positives.
    Args:
        df: DataFrame with a label column
        label_col: Name of the label column
        ratio: Negative:positive ratio
        random_state: For reproducibility
    Returns:
        Subsetted DataFrame
    """
    pos_df = df[df[label_col] == 1]
    neg_df = df[df[label_col] == 0]
    n_pos = len(pos_df)
    n_neg = min(len(neg_df), n_pos * ratio)
    neg_df = neg_df.sample(n=n_neg, random_state=random_state)
    return (
        pd.concat([pos_df, neg_df])
        .sample(frac=1, random_state=random_state)
        .reset_index(drop=True)
    )

In [5]:
def cluster_positives(
    df, bitscore_file, bitscore_cutoff=100, id_col="id", label_col="label"
):
    pos_ids = set(df[df[label_col] == 1][id_col])

    # Load diamond results (parquet)
    diamond = pd.read_parquet(bitscore_file)
    # If needed, rename columns here:
    # diamond = diamond.rename(columns={"col1": "qseqid", "col2": "sseqid", "col3": "bitscore"})

    # Filter for bitscore cutoff and only positive IDs
    diamond = diamond[
        (diamond["bitscore"] >= bitscore_cutoff)
        & (diamond["qseqid"].isin(pos_ids))
        & (diamond["sseqid"].isin(pos_ids))
    ]

    # Build graph
    G = nx.Graph()
    G.add_edges_from(diamond[["qseqid", "sseqid"]].itertuples(index=False, name=None))

    # Assign cluster IDs
    cluster_mapping = {}
    for i, component in enumerate(nx.connected_components(G)):
        for node in component:
            cluster_mapping[node] = f"cluster_{i}"

    # Assign unconnected positives to their own cluster
    for pid in pos_ids:
        if pid not in cluster_mapping:
            cluster_mapping[pid] = f"cluster_single_{pid}"

    return cluster_mapping

In [6]:
def prepare_data(
    df: pd.DataFrame,
    feature_cols: list,
    cluster_mapping: Dict[str, str],
    label_col: str = "label",
    id_col: str = "id",
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Prepare data for training by separating features and labels.

    Args:
        df: DataFrame containing features and labels
        feature_cols: List of feature column names
        cluster_mapping: Dictionary mapping sequence IDs to cluster IDs
        label_col: Name of the label column
        id_col: Name of the ID column

    Returns:
        X: Feature matrix
        y: Label array
        ids: Array of sequence IDs
        groups: Array of cluster IDs for each sequence
    """
    X = df[feature_cols].values
    y = df[label_col].values
    ids = df[id_col].values

    # Get cluster IDs for each sequence
    groups = np.array([cluster_mapping.get(str(id_), "unknown") for id_ in ids])

    return X, y, ids, groups

In [7]:
def split_data(
    X: np.ndarray,
    y: np.ndarray,
    ids: np.ndarray,
    groups: np.ndarray,
    test_size: float = 0.2,
    val_size: float = 0.2,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Split data into train, validation, and test sets while keeping related sequences together.

    Args:
        X: Feature matrix
        y: Label array
        ids: Array of sequence IDs
        groups: Array of cluster IDs for each sequence
        test_size: Proportion of data to use for testing
        val_size: Proportion of data to use for validation
    """
    # First split: separate test set
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    train_val_idx, test_idx = next(gss.split(X, y, groups=groups))

    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y[train_val_idx], y[test_idx]
    groups_train_val = groups[train_val_idx]

    # Second split: separate validation set from training set
    val_size_adjusted = val_size / (
        1 - test_size
    )  # Adjust val_size to account for test set
    gss = GroupShuffleSplit(n_splits=1, test_size=val_size_adjusted, random_state=42)
    train_idx, val_idx = next(
        gss.split(X_train_val, y_train_val, groups=groups_train_val)
    )

    X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
    y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

    return X_train, X_val, X_test, y_train, y_val, y_test


# first split: test 20%
# train:val = 8:2

In [8]:
def train_model(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_val: np.ndarray,
    y_val: np.ndarray,
    model_params: Dict[str, Any] = None,
) -> Tuple[Any, StandardScaler]:
    """Train an XGBoost classifier with optional hyperparameters."""
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Set default parameters if none provided
    if model_params is None:
        model_params = {
            "n_estimators": 200,
            "max_depth": 6,
            "learning_rate": 0.1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "min_child_weight": 1,
            "scale_pos_weight": 1,
            "random_state": 42,
        }

    # Calculate scale_pos_weight based on class imbalance
    n_neg = np.sum(y_train == 0)
    n_pos = np.sum(y_train == 1)
    model_params["scale_pos_weight"] = n_neg / n_pos

    # Train model
    model = XGBClassifier(eval_metric=["logloss", "auc"], **model_params)
    model.fit(
        X_train_scaled,
        y_train,
        eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)],
        verbose=False,
        # early_stopping_rounds=50,
    )
    evals_result = model.evals_result()
    return model, scaler, evals_result

In [9]:
def evaluate_model(
    model: Any,
    scaler: StandardScaler,
    X: np.ndarray,
    y: np.ndarray,
    set_name: str = "",
    threshold: float = 0.5,
) -> Dict[str, float]:
    """Evaluate model performance on a dataset."""

    # no need for scaling!

    X_scaled = scaler.transform(X)
    y_pred_proba = model.predict_proba(X_scaled)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Calculate MCC
    def matthews(y_true, y_pred):
        from math import sqrt

        """
            P  = Total number of positives
            N  = Total number of negatives
            Tp = number of true positives
            Fp = number of false positives
        """
        if type(y_true) == pd.Series:
            y_true = y_true.values

        P = len([x for x in y_true if x == 1])
        N = len([x for x in y_true if x == 0])

        Tp, Fp = 0, 0
        for i in range(len(y_true)):
            if y_true[i] == 1 and y_pred[i] == 1:
                Tp += 1
            elif y_true[i] == 0 and y_pred[i] == 1:
                Fp += 1

        Tn = N - Fp
        Fn = P - Tp

        try:
            mcc = (Tp * Tn - Fp * Fn) / sqrt(
                (Tn + Fn) * (Tn + Fp) * (Tp + Fn) * (Tp + Fp)
            )
        except ZeroDivisionError:
            mcc = 0
        return (mcc, f"P: {P:_} Tp: {Tp:_} Fp: {Fp:_} N: {N:_} Tn: {Tn:_} Fn: {Fn:_}")

    # Get MCC and confusion matrix values
    mcc, confusion_str = matthews(y, y_pred)

    metrics = {
        f"{set_name}_accuracy": accuracy_score(y, y_pred),
        f"{set_name}_precision": precision_score(y, y_pred),
        f"{set_name}_recall": recall_score(y, y_pred),
        f"{set_name}_f1": f1_score(y, y_pred),
        f"{set_name}_roc_auc": roc_auc_score(y, y_pred_proba),
        f"{set_name}_mcc": mcc,
    }

    # Print metrics
    print(f"\nMetrics for {set_name} (threshold={threshold}):")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    # Print confusion matrix values
    print(f"\nConfusion Matrix Values:")
    print(confusion_str)

    return metrics

In [10]:
# def plot_feature_importance(model: Any, feature_cols: list, top_n: int = 20):
#     """Plot feature importance from XGBoost model."""
#     importance_scores = model.feature_importances_
#     indices = np.argsort(importance_scores)[::-1]

#     plt.figure(figsize=(12, 6))
#     plt.title("Feature Importances")
#     plt.bar(range(top_n), importance_scores[indices[:top_n]])
#     plt.xticks(range(top_n), [feature_cols[i] for i in indices[:top_n]], rotation=90)
#     plt.tight_layout()
#     plt.show()

#     # Print top N most important features
#     print(f"\nTop {top_n} most important features:")
#     for i in range(top_n):
#         print(f"{feature_cols[indices[i]]}: {importance_scores[indices[i]]:.4f}")

In [11]:
def save_model(
    model: Any, scaler: StandardScaler, feature_cols: list, function_name: str
):
    """Save the trained model and scaler."""
    # Create models directory if it doesn't exist
    os.makedirs("../models", exist_ok=True)

    # Create a dictionary containing all necessary components
    model_data = {"model": model, "scaler": scaler, "feature_cols": feature_cols}

    # Save the model data
    joblib.dump(model_data, f"../models/{function_name}_predictor.joblib")

In [12]:
def train_function_predictor(function_name: str, model_params: Dict[str, Any] = None):
    """Main training pipeline for a specific protein function."""
    # # Load cluster mapping
    # with open("../dataset/protein_cluster_mapping.json", "r") as f:
    #     cluster_mapping = json.load(f)

    # get df using function name
    df = get_df(function_name)
    df = subset_negatives(df, ratio=5)

    # Cluster positives using diamond results
    bitscore_file = "../dataset/unique_diamond_results.parquet"
    cluster_mapping = cluster_positives(df, bitscore_file, bitscore_cutoff=100)

    # Assign dummy group to negatives
    for idx, row in df.iterrows():
        if row["label"] == 0:
            cluster_mapping[row["id"]] = f"neg_{row['id']}"

    # Get feature columns (excluding 'id' and 'label')
    feature_cols = [col for col in df.columns if col not in ["id", "label"]]

    # Prepare data
    X, y, ids, groups = prepare_data(df, feature_cols, cluster_mapping)

    # Split data
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y, ids, groups)

    # Train model
    model, scaler, evals_result = train_model(
        X_train, y_train, X_val, y_val, model_params
    )

    train_metric = evals_result["validation_0"]["logloss"]
    val_metric = evals_result["validation_1"]["logloss"]

    plt.figure()
    plt.plot(train_metric, label="Train Logloss")
    plt.plot(val_metric, label="Val Logloss")
    plt.xlabel("Boosting Round")
    plt.ylabel("Logloss")
    plt.title(f"Learning Curve: {function_name}")
    plt.legend()
    os.makedirs(f"../results/predictor/learning_curves", exist_ok=True)
    plt.savefig(
        f"../results/predictor/learning_curves/{function_name}_learning_curve.png"
    )
    plt.close()

    # Collect results in a list of strings
    results_lines = []

    # Feature importances
    importance_scores = model.feature_importances_
    indices = np.argsort(importance_scores)[::-1]
    results_lines.append("Top 20 most important features:")
    for i in range(20):
        results_lines.append(
            f"{feature_cols[indices[i]]}: {importance_scores[indices[i]]:.4f}"
        )
    results_lines.append("")

    # Train metrics
    train_metrics = evaluate_model(model, scaler, X_train, y_train, "train")
    results_lines.append("=== Training Set ===")
    for k, v in train_metrics.items():
        results_lines.append(f"{k}: {v}")
    results_lines.append("")

    # Validation metrics
    val_metrics = evaluate_model(model, scaler, X_val, y_val, "val")
    results_lines.append("=== Validation Set ===")
    for k, v in val_metrics.items():
        results_lines.append(f"{k}: {v}")
    results_lines.append("")

    # Test metrics for all thresholds
    results_lines.append("=== Test Set ===")
    thresholds = np.linspace(0.01, 1.0, 100)
    best_mcc = -float("inf")
    best_threshold = None

    for threshold in thresholds:
        test_metrics = evaluate_model(model, scaler, X_test, y_test, "test", threshold)
        results_lines.append(f"Threshold: {threshold:.2f}")
        for k, v in test_metrics.items():
            results_lines.append(f"{k}: {v}")
        results_lines.append("")
        # Track best MCC
        mcc = test_metrics.get("test_mcc", -float("inf"))
        if isinstance(
            mcc, tuple
        ):  # If your evaluate_model returns (mcc, confusion_str)
            mcc = mcc[0]
        if mcc > best_mcc:
            best_mcc = mcc
            best_threshold = threshold

    # Add the best MCC summary line
    results_lines.append(f"Best MCC: {best_mcc:.4f} at threshold {best_threshold:.2f}")
    results_lines.append("")

    # Save all results to file
    os.makedirs("../results/predictor", exist_ok=True)
    result_file = f"../results/predictor/{function_name}.txt"
    with open(result_file, "w") as f:
        f.write("\n".join(results_lines))

    # Save model
    save_model(model, scaler, feature_cols, function_name)

# execution

In [15]:
model_params = {
    "n_estimators": 10000,
    "max_depth": 8,
    "learning_rate": 0.05,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "min_child_weight": 2,
    "random_state": 42,
    "early_stopping_rounds": 100,
}

train_function_predictor("connector", model_params)


Metrics for train (threshold=0.5):
train_accuracy: 0.9996
train_precision: 0.9979
train_recall: 1.0000
train_f1: 0.9990
train_roc_auc: 1.0000
train_mcc: 0.9987

Confusion Matrix Values:
P: 10_956 Tp: 10_956 Fp: 23 N: 47_287 Tn: 47_264 Fn: 0

Metrics for val (threshold=0.5):
val_accuracy: 0.9180
val_precision: 0.8266
val_recall: 0.4639
val_f1: 0.5943
val_roc_auc: 0.9039
val_mcc: 0.5813

Confusion Matrix Values:
P: 2_343 Tp: 1_087 Fp: 228 N: 15_760 Tn: 15_532 Fn: 1_256

Metrics for test (threshold=0.01):
test_accuracy: 0.6310
test_precision: 0.2655
test_recall: 0.9793
test_f1: 0.4177
test_roc_auc: 0.9511
test_mcc: 0.3801

Confusion Matrix Values:
P: 2_464 Tp: 2_413 Fp: 6_676 N: 15_768 Tn: 9_092 Fn: 51

Metrics for test (threshold=0.02):
test_accuracy: 0.7530
test_precision: 0.3504
test_recall: 0.9692
test_f1: 0.5147
test_roc_auc: 0.9511
test_mcc: 0.4864

Confusion Matrix Values:
P: 2_464 Tp: 2_388 Fp: 4_428 N: 15_768 Tn: 11_340 Fn: 76

Metrics for test (threshold=0.03):
test_accuracy: 0

/home/spica/Repos/pici-predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# example for testing early stopping
# import xgboost

# print("XGBoost version:", xgboost.__version__)

# from xgboost import XGBClassifier
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split

# X, y = load_iris(return_X_y=True)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# model = XGBClassifier(n_estimators=10)
# model.fit(
#     X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=5, verbose=True
# )